<a href="https://colab.research.google.com/github/Anoj-07/Anoj-07/blob/main/English_To_Garo_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/en_gr (1).csv')

In [ ]:
df.head()

,English,Garo
0,Everything,Pilak bostuan
1,Nowhere,Darangchiba
2,Somewhere,Jechiba
3,Everywhere,Gimikchian
4,Everytime,pangnan


#Cleaning Data Set

In [ ]:
# converting every letter to lower case
df['English'] = df['English'].apply(lambda x: str(x).lower())
df['Garo'] = df['Garo'].apply(lambda x: str(x).lower())

In [ ]:
# removing apostrophe from the sentences
df['English'] = df['English'].apply(lambda x: re.sub("'","",x))
df['Garo'] = df['Garo'].apply(lambda x: re.sub("'","",x))

In [ ]:
exclude = set(string.punctuation)
# removing all the punctuations
df['English'] = df['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['Garo'] = df['Garo'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# removing digits from the sentences
digit = str.maketrans('','',string.digits)
df['English'] = df['English'].apply(lambda x: x.translate(digit))
df['Garo'] = df['Garo'].apply(lambda x: x.translate(digit))

#Using pretrained model and then finetunig

In [ ]:
# using pretrained model and then finetunig it on our dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

#model Optimizer

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0001)

#Model Train

In [ ]:
def model_train():
    model.train()
    losses = 0
    X = df['English']
    y = df['Garo']
    max_epochs = 15
    n_batches = 32
    for epoch in tqdm(range(max_epochs)):
        for i in tqdm(range(n_batches)):
            # making batches
            local_X, local_y = X[i*n_batches:(i+1)*n_batches,], y[i*n_batches:(i+1)*n_batches,]
            # preparing the data according to the model input
            batch = tokenizer.prepare_seq2seq_batch(list(local_X),list(local_y),return_tensors='pt')
            output = model(**batch)
            # loss can be taken directly from the model output
            loss = output.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses = losses+loss
    average = losses/len(df)
    print('Loss: ' + str(average) )

    return model

In [ ]:
model = model_train()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is depr

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Loss: tensor(0.1222, grad_fn=<DivBackward0>)


In [ ]:
a = model.generate(**tokenizer.prepare_seq2seq_batch([' i will see you tomorrow'],return_tensors='pt'))
tokenizer.batch_decode(a)

#pad sequences

['<pad> knalo nangko gronggen</s>']

In [ ]:
torch.save(model , 'model.pkl')

In [ ]:
# def model_train():
#     model.train()
#     losses = 0
#     X = df['English'].tolist()  # Convert DataFrame column to list
#     y = df['Garo'].tolist()  # Convert DataFrame column to list
#     max_epochs = 15
#     batch_size = 32
#     n_batches = len(df) // batch_size  # Calculate number of batches
#     for epoch in tqdm(range(max_epochs)):
#         for i in tqdm(range(n_batches)):
#             # Making batches
#             start_idx = i * batch_size
#             end_idx = (i + 1) * batch_size
#             local_X, local_y = X[start_idx:end_idx], y[start_idx:end_idx]
#             # Preparing the data according to the model input
#             batch = tokenizer.prepare_seq2seq_batch(local_X, local_y, return_tensors='pt', padding=True, truncation=True)
#             output = model(**batch)
#             # Loss can be taken directly from the model output
#             loss = output.loss
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             losses += loss.item()
#     average_loss = losses / (max_epochs * n_batches)
#     print('Average Loss:', average_loss)

#     return model

# trained_model = model_train()

#TO FIND BLUE SCORE

#Pre-train Tokenizer

In [ ]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import corpus_bleu

# Load the saved model
try:
    model = torch.load('model.pkl')
except RuntimeError as e:
    if "failed finding central directory" in str(e):
        print("Error loading model: Corrupted or inaccessible model.pkl file.")
        # Try a different serialization method or file path here
    else:
        raise e

# Load the dataset
df = pd.read_csv('/content/en_gr (1).csv')

# Preprocess the dataset
df['English'] = df['English'].apply(lambda x: str(x).lower())
df['Garo'] = df['Garo'].apply(lambda x: str(x).lower())
df['English'] = df['English'].apply(lambda x: re.sub("'", "", x))
df['Garo'] = df['Garo'].apply(lambda x: re.sub("'", "", x))
exclude = set(string.punctuation)
df['English'] = df['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['Garo'] = df['Garo'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
digit = str.maketrans('', '', string.digits)
df['English'] = df['English'].apply(lambda x: x.translate(digit))
df['Garo'] = df['Garo'].apply(lambda x: x.translate(digit))

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Generate translations
generated_translations = []
for source_sentence in tqdm(df['English']):
    # Generate translation
    translation = model.generate(**tokenizer.prepare_seq2seq_batch([source_sentence], return_tensors='pt'))
    # Decode the translation
    translated_text = tokenizer.batch_decode(translation, skip_special_tokens=True)[0]
    generated_translations.append(translated_text)

# Prepare reference translations
reference_translations = df['Garo'].tolist()




/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  0%|          | 0/3318 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt sentence tokenizer model (one-time download)
nltk.download('punkt')

# Tokenize generated and reference translations
reference_tokenized = [[token for token in word_tokenize(ref)] for ref in reference_translations]
generated_tokenized = [[token for token in word_tokenize(trans)] for trans in generated_translations]

# Calculate BLEU score
bleu_score = corpus_bleu(reference_tokenized, generated_tokenized)
print("BLEU Score:", bleu_score)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU Score: 4.466715240105322e-233


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

#Random Text

In [ ]:
import torch

# Assuming you have a trained model (`model`) from the previous code

def predict_garo(english_sentence):
  """
  Predicts a Garo sentence given an English sentence using the trained model.

  Args:
      english_sentence (str): The English sentence to translate.

  Returns:
      str: The predicted Garo sentence.
  """

  # Preprocess the English sentence (e.g., convert to lowercase, remove punctuation)
  # You might want to implement the same preprocessing steps you used during training.
  processed_english_sentence = english_sentence.lower()  # Example preprocessing

  # Tokenize the English sentence
  input_ids = tokenizer.encode(processed_english_sentence, return_tensors="pt")

  # Generate the Garo sentence using beam search
  with torch.no_grad():
    output = model.generate(input_ids, max_length=50, num_beams=5)  # Adjust max_length and num_beams as needed

  # Decode the generated tokens into text
  predicted_garo_sentence = tokenizer.decode(output[0], skip_special_tokens=True)

  return predicted_garo_sentence

In [ ]:
import transformers  # Assuming you're using transformers for tokenization
import random
# ... other code ...

# Define the english_tokenizer outside the function (if needed from outside)
english_tokenizer = transformers.AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")  # Or your specific English tokenizer

def get_random_sentence(df, english_column='English', garo_column='Garo', english_tokenizer=None, garo_tokenizer=None):
  """
  Retrieves a random sentence from the DataFrame, optionally tokenizes, and includes a prediction placeholder.

  Args:
      df (pandas.DataFrame): The DataFrame containing English and Garo sentences.
      english_column (str, optional): The column name containing English sentences. Defaults to 'English'.
      garo_column (str, optional): The column name containing Garo sentences. Defaults to 'Garo'.
      english_tokenizer (transformers.PreTrainedTokenizer, optional): The tokenizer for English sentences.
      garo_tokenizer (transformers.PreTrainedTokenizer, optional): The tokenizer for Garo sentences.

  Returns:
      tuple: A tuple containing the random English sentence, predicted Garo sentence (placeholder), and the true Garo sentence (all optionally tokenized).
  """

  # Get random index within DataFrame length
  random_index = random.randint(0, len(df) - 1)

  # Retrieve English and Garo sentences
  english_sentence = df.loc[random_index, english_column]
  garo_sentence = df.loc[random_index, garo_column]

  # Tokenize if english_tokenizer is provided
  if english_tokenizer:
    english_sentence = english_tokenizer.sequences_to_texts([[english_sentence]])[0]

  # Optionally include prediction (replace with your prediction function)
  predicted_garo_sentence = predict_garo(english_sentence) # Replace with your prediction function call(english_sentence)

  # Return the results within the function
  return english_sentence, predicted_garo_sentence, garo_sentence


In [ ]:
# Assuming you have a DataFrame 'df' with 'English' and 'Garo' columns
# (and optionally, initialized tokenizers)
for _ in range(10):
  english_sentence, predicted_garo_sentence, garo_sentence = get_random_sentence(df)

  print(f"English sentence: {english_sentence}")
  print(f"Predicted Garo sentence: {predicted_garo_sentence}")  # Replace with actual prediction

  print("----------------")


English sentence: leaverelease
Predicted Garo sentence: sengbo
----------------
English sentence: cook
Predicted Garo sentence: kakket kakketma
----------------
English sentence: small pox
Predicted Garo sentence: pitcheng
----------------
English sentence: watch 
Predicted Garo sentence: ghori
----------------
English sentence: he has a headache 
Predicted Garo sentence: ua kaki a kenna dongama
----------------
English sentence: oat 
Predicted Garo sentence: oatchi
----------------
English sentence: relative
Predicted Garo sentence: madrang
----------------
English sentence: drink milk
Predicted Garo sentence: dut ringbo
----------------
English sentence: thumb 
Predicted Garo sentence: kosakchi
----------------
English sentence: cousin brother
Predicted Garo sentence: mosa pani noni de
----------------


In [ ]:
for _ in range(5):
  english_sentence, predicted_garo_sentence, garo_sentence = get_random_sentence(df)

  print(f"English sentence: {english_sentence}")
  print(f"Predicted Garo sentence: {predicted_garo_sentence}")  # Replace with actual prediction
  print(f"True Garo sentence: {garo_sentence}")
  print("----------------")

English sentence: i have to meet my son
Predicted Garo sentence: anga angni depante ko grongna nangen
True Garo sentence: anga angni depante ko grongna nangen
----------------
English sentence: they halt at staying houses
Predicted Garo sentence: uamang dongparagen
True Garo sentence: uamang dak banglarango machaka
----------------
English sentence: cream 
Predicted Garo sentence: makon
True Garo sentence: cream 
----------------
English sentence: red pepper 
Predicted Garo sentence: gitchak
True Garo sentence: gulmorich
----------------
English sentence:  everything 
Predicted Garo sentence: pilak bostuan
True Garo sentence: pilak bostuan
----------------
